# 뉴스 데이터 전처리

이 노트북은 크롤링한 뉴스 데이터를 BERT 모델 학습을 위해 전처리하는 과정을 담고 있습니다.

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

## 1. 데이터 로드 및 병합

In [2]:
# 데이터 경로 설정
data_path = Path('../data')

# 모든 CSV 파일 리스트 가져오기
csv_files = list(data_path.glob('*.csv'))

# 데이터프레임 리스트 생성
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    # 언론사 이름 추출 (파일명에서)
    press_name = file.stem.split('_')[0]
    df['press'] = press_name
    dfs.append(df)

# 모든 데이터프레임 병합
combined_df = pd.concat(dfs, ignore_index=True)
print(f"총 {len(combined_df)} 개의 기사가 있습니다.")

총 85698 개의 기사가 있습니다.


In [16]:
# 이미 병합했다면
combined_df = pd.read_csv('../data/전체통합.csv')
print(f"총 {len(combined_df)} 개의 기사가 있습니다.")

총 85698 개의 기사가 있습니다.


## 2. 텍스트 전처리

In [6]:
def clean_text1(text):
    if not isinstance(text, str):
        return ""
    
    # URL 제거
    text = re.sub(r'https?://\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # 줄바꿈/개행 문자를 공백으로 변환
    text = re.sub(r'[\n\r]+', ' ', text)
    
    # 특수문자 제거 (한글, 영문자, 숫자만 남김)
    text = re.sub(r'[^\w\s가-힣]', ' ', text)
    
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 문장 시작과 끝의 공백 제거
    text = text.strip()
    
    return text

In [5]:
def clean_text(text):
    if not isinstance(text, str):
        return ""

    # (1) 기자/출처 제거
    text = re.sub(r'\([^\)]+기자\)', '', text)
    text = re.sub(r'[가-힣]{2,4}\s?기자(입니다)?', '', text)
    text = re.sub(r'기자\s*=\s*', '', text)
    text = re.sub(r'사진[=:\-]\s*[^,\n]+', '', text)
    text = re.sub(r'(연합뉴스|뉴스1|뉴시스|SBS|KBS|MBC|JTBC|YTN)', '', text)

    # (2) HTML 태그 및 URL 제거
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'https?://\S+', '', text)

    # (3) 괄호/대괄호 제거
    text = re.sub(r'\(서울=.*?\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)

    # (4) 공통 및 언론사 클리셰 제거
    noise_patterns = [
        r'무단 전재 및 재배포 금지',
        r'해당 기사.*?무단 전재.*?금지합니다',
        r'자세한 내용은.*?확인하십시오',
        r'본 기사.*?무단 전재.*?금지',
        r'ⓒ 오마이뉴스.*',
        r'오마이뉴스\s*ⓒ.*',
        r'조선닷컴.*',
        r'경향신문.*?재배포 금지',
        r'중앙일보.*?무단 전재.*',
        r'한국일보.*?저작권.*',
        r'SBS.*?All rights reserved',
        r'All rights reserved.*',
        r'사진[=:\-]\s*[^,\n]+',
        r'\(사진=\s*SBS.*?\)',
        r'경향신문\s*무단 전재.*',
        r'동아일보\s*무단 전재.*',
        r'중앙일보\s*재배포.*',
        r'한국일보\s*무단.*',
        r'뉴스1',
        r'연합뉴스',
        r'뉴시스',
        r'기자\s*입력\s*\d{4}',
        r'기자\s*승인\s*\d{4}',
        r'중앙일보\s*제공',
    ]
    for pattern in noise_patterns:
        text = re.sub(pattern, '', text)

    # (5) 특수문자 정리 (의미 있는 기호는 유지)
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s가-힣%℃·\-]', '', text)

    return text.strip()

In [17]:
# 결측치 제거
combined_df = combined_df.dropna(subset=['title', 'content'])

# 제목과 본문 전처리
combined_df['title_cleaned'] = combined_df['title'].apply(clean_text1)
combined_df['title_cleaned1'] = combined_df['title'].apply(clean_text)
combined_df['content_cleaned'] = combined_df['content'].apply(clean_text)

# 제목과 본문 결합
combined_df['text'] = combined_df['title_cleaned'] + ' ' + combined_df['content_cleaned']

# 너무 짧은 기사 제거 (예: 100자 미만)
combined_df = combined_df[combined_df['text'].str.len() >= 100]

# 중복 기사 제거 (제목과 본문이 동일한 경우)
combined_df = combined_df.drop_duplicates(subset=['title_cleaned', 'content_cleaned'])

# 중복 제거 후 데이터 수 확인
print(f"중복 제거 후 남은 기사 수: {len(combined_df)}")

중복 제거 후 남은 기사 수: 78973


In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78927 entries, 0 to 85697
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            78927 non-null  object
 1   content          78927 non-null  object
 2   press            78927 non-null  object
 3   url              78927 non-null  object
 4   created_date     78927 non-null  object
 5   modified_date    59553 non-null  object
 6   journalist       67561 non-null  object
 7   comment_count    78927 non-null  int64 
 8   related_titles   78425 non-null  object
 9   related_urls     78425 non-null  object
 10  title_cleaned1   78927 non-null  object
 11  title_cleaned    78927 non-null  object
 12  content_cleaned  78927 non-null  object
 13  text             78927 non-null  object
dtypes: int64(1), object(13)
memory usage: 9.0+ MB


In [18]:
# 전처리된 통합 데이터셋 저장
combined_df.to_csv("../data/전체통합_전처리.csv", index=False)

In [19]:
import pandas as pd

# 파일 경로
path_labeled = '../data/정당_관점_라벨링_최종.csv'
path_cleaned = '../data/전체통합_전처리.csv'

# CSV 로딩
df_labeled = pd.read_csv(path_labeled)
df_cleaned = pd.read_csv(path_cleaned)

In [20]:
df_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title_cleaned    1000 non-null   object
 1   content_cleaned  1000 non-null   object
 2   party            1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


In [21]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78973 entries, 0 to 78972
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            78973 non-null  object
 1   content          78973 non-null  object
 2   press            78973 non-null  object
 3   url              78973 non-null  object
 4   created_date     78973 non-null  object
 5   modified_date    59595 non-null  object
 6   journalist       67571 non-null  object
 7   comment_count    78973 non-null  int64 
 8   related_titles   78471 non-null  object
 9   related_urls     78471 non-null  object
 10  title_cleaned    78973 non-null  object
 11  title_cleaned1   78973 non-null  object
 12  content_cleaned  78973 non-null  object
 13  text             78973 non-null  object
dtypes: int64(1), object(13)
memory usage: 8.4+ MB


In [27]:
df_cleaned.drop_duplicates(subset=['title', 'content'], inplace=True)

In [28]:
# title 기준으로 merge (주의: title 컬럼이 서로 이름이 다를 수 있음)
# 예: labeled 파일에는 title, cleaned 파일에는 title_cleaned1
df_merged = pd.merge(df_labeled, df_cleaned, on="title_cleaned", how="left", suffixes=('_labeled', '_cleaned'))

In [30]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title_cleaned            1034 non-null   object 
 1   content_cleaned_labeled  1034 non-null   object 
 2   party                    1034 non-null   object 
 3   title                    1033 non-null   object 
 4   content                  1033 non-null   object 
 5   press                    1033 non-null   object 
 6   url                      1033 non-null   object 
 7   created_date             1033 non-null   object 
 8   modified_date            809 non-null    object 
 9   journalist               876 non-null    object 
 10  comment_count            1033 non-null   float64
 11  related_titles           1028 non-null   object 
 12  related_urls             1028 non-null   object 
 13  title_cleaned1           1033 non-null   object 
 14  content_cleaned_cleaned 

In [31]:
# 최신 전처리된 title과 content로 갱신
df_merged["title_cleaned"] = df_merged["title_cleaned1"]
df_merged["content_cleaned"] = df_merged["content_cleaned_cleaned"]

# 불필요한 중복 컬럼 제거
df_merged = df_merged.drop(columns=["title_cleaned1", "content_cleaned_labeled", "content_cleaned_cleaned"])

In [34]:
df_merged.drop_duplicates(subset=['title_cleaned', 'content_cleaned'], inplace=True)

In [38]:
df_merged.dropna(subset=['title_cleaned', 'content_cleaned'],  inplace=True)

In [39]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1025 entries, 0 to 1033
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title_cleaned    1025 non-null   object 
 1   party            1025 non-null   object 
 2   title            1025 non-null   object 
 3   content          1025 non-null   object 
 4   press            1025 non-null   object 
 5   url              1025 non-null   object 
 6   created_date     1025 non-null   object 
 7   modified_date    801 non-null    object 
 8   journalist       870 non-null    object 
 9   comment_count    1025 non-null   float64
 10  related_titles   1020 non-null   object 
 11  related_urls     1020 non-null   object 
 12  text             1025 non-null   object 
 13  content_cleaned  1025 non-null   object 
dtypes: float64(1), object(13)
memory usage: 120.1+ KB


In [43]:
# 필요한 컬럼만 선택 및 순서 지정
final_columns = ["title_cleaned", "content_cleaned", "text", "press", "created_date", "party"]
df_final = df_merged[final_columns]

# 저장
df_final.to_csv("../data/정당_관점_라벨링_최종_업데이트.csv", index=False)

In [44]:
import pandas as pd

df = pd.read_csv('../data/전체통합_전처리.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78973 entries, 0 to 78972
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            78973 non-null  object
 1   content          78973 non-null  object
 2   press            78973 non-null  object
 3   url              78973 non-null  object
 4   created_date     78973 non-null  object
 5   modified_date    59595 non-null  object
 6   journalist       67571 non-null  object
 7   comment_count    78973 non-null  int64 
 8   related_titles   78471 non-null  object
 9   related_urls     78471 non-null  object
 10  title_cleaned    78973 non-null  object
 11  title_cleaned1   78973 non-null  object
 12  content_cleaned  78973 non-null  object
 13  text             78973 non-null  object
dtypes: int64(1), object(13)
memory usage: 8.4+ MB


In [ ]:
df['title_cleaned'] = df['title_cleaned1']
df = df[['title_cleaned', 'content_cleaned', 'text', 'press', 'url', 'created_date', 'modified_date', 'journalist', 'comment_count', 'related_titles', 'related_urls']]
df.to_csv('../data/전체통합_전처리.csv', index=False)

In [46]:
df.to_csv('../data/전체통합_전처리.csv', index=False)

In [48]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
df = pd.read_csv('../data/전체통합_전처리.csv')

# 전체 데이터 수 확인
total_rows = len(df)
print(f"전체 데이터 수: {total_rows}")

전체 데이터 수: 78973


In [49]:
# 랜덤하게 데이터 섞기
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 각 그룹의 크기 설정
group1_size = 3000
group2_size = 5000
group3_size = 10000
group4_size = total_rows - (group1_size + group2_size + group3_size)

# 데이터 분할
group1 = df.iloc[:group1_size]
group2 = df.iloc[group1_size:group1_size + group2_size]
group3 = df.iloc[group1_size + group2_size:group1_size + group2_size + group3_size]
group4 = df.iloc[group1_size + group2_size + group3_size:]

# 각 그룹을 CSV 파일로 저장
group1.to_csv('../model/data/전체1.csv', index=False)
group2.to_csv('../model/data/전체2.csv', index=False)
group3.to_csv('../model/data/전체3.csv', index=False)
group4.to_csv('../model/data/전체4.csv', index=False)

# 각 그룹의 크기 출력
print(f"전체 1 크기: {len(group1)}")
print(f"전체 2 크기: {len(group2)}")
print(f"전체 3 크기: {len(group3)}")
print(f"전체 4 크기: {len(group4)}")

전체 1 크기: 3000
전체 2 크기: 5000
전체 3 크기: 10000
전체 4 크기: 60973
